![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.1.Clinical_Deidentification_in_German.ipynb)

# Clinical Deidentification in German

**Protected Health Information**:

Individual’s past, present, or future physical or mental health or condition
provision of health care to the individual
past, present, or future payment for the health care
Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [2]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 46.9 MB/s 
     |████████████████████████████████| 148 kB 398 kB/s 
     |████████████████████████████████| 142 kB 4.4 MB/s 


In [3]:
import sys
import os
import json
import pandas as pd
import string
import numpy as np

import sparknlp
import sparknlp_jsl

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import ResourceDownloader
from sparknlp.util import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(SECRET, params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.4.2


# 1. German NER Deidentification Models 

|index|model|index|model|
|-----:|:-----|-----:|:-----|
| 1| [ner_deid_generic]()| 2| [ner_deid_subentity]()|
| 3| [ner_deid_generic_pipeline]()| 4| [ner_deid_subentity_pipeline]()|

### Creating pipeline

In [ ]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d","de","clinical/models")\
    .setInputCols(["document","token"])\
	  .setOutputCol("embeddings")

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


## 1.1. NER Deid Generic

**`ner_deid_generic`** extracts:
- Name
- Profession
- Age
- Date
- Contact (Telephone numbers, FAX numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Employment information)
- Id (Social Security numbers, Medical record numbers, Internet protocol addresses)



In [ ]:
ner_generic = MedicalNerModel.pretrained("ner_deid_generic", "de", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_generic")

ner_converter_generic = NerConverter()\
    .setInputCols(["sentence","token","ner_deid_generic"])\
    .setOutputCol("ner_chunk_generic")

ner_deid_generic download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [ ]:
ner_generic.getClasses()

['O',
 'I-LOCATION',
 'B-DATE',
 'I-NAME',
 'B-LOCATION',
 'I-DATE',
 'B-ID',
 'B-AGE',
 'B-CONTACT',
 'B-PROFESSION',
 'B-NAME']

## 1.2. NER Deid Subentity

**`ner_deid_subentity`** extracts:

- Patient
- Doctor
- Hospital
- Date
- Organization
- City
- Street
- User Name
- Profession
- Phone
- Country
- Age

In [ ]:
ner_subentity = MedicalNerModel.pretrained("ner_deid_subentity", "de", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_subentity")

ner_converter_subentity = NerConverter()\
    .setInputCols(["sentence", "token", "ner_deid_subentity"])\
    .setOutputCol("ner_chunk_subentity")

ner_deid_subentity download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [ ]:
ner_subentity.getClasses()

['O',
 'B-ORGANIZATION',
 'I-DOCTOR',
 'B-DOCTOR',
 'B-USERNAME',
 'I-CITY',
 'I-DATE',
 'B-COUNTRY',
 'B-PROFESSION',
 'I-STREET',
 'I-PATIENT',
 'B-PHONE',
 'B-CITY',
 'B-HOSPITAL',
 'B-DATE',
 'B-STREET',
 'B-PATIENT',
 'I-ORGANIZATION',
 'I-HOSPITAL',
 'B-AGE',
 'I-COUNTRY']

## 1.4. Pipeline

In [ ]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_generic,
      ner_converter_generic,
      ner_subentity,
      ner_converter_subentity,
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:
text = """Michael Berger wird am Morgen des 12 Dezember 2018 ins St. Elisabeth-Krankenhaus in Bad Kissingen eingeliefert. Herr Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen."""

text_df = spark.createDataFrame([[text]]).toDF("text")

result = model.transform(text_df)


### Results for `ner_subentity`

In [ ]:
result.select(F.explode(F.arrays_zip('ner_chunk_subentity.result', 'ner_chunk_subentity.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+--------------------------------------+---------+
|chunk                                 |ner_label|
+--------------------------------------+---------+
|Michael Berger                        |PATIENT  |
|12 Dezember 2018                      |DATE     |
|Elisabeth-Krankenhaus in Bad Kissingen|HOSPITAL |
|Berger                                |PATIENT  |
|76                                    |AGE      |
+--------------------------------------+---------+



### Results for `ner_generic`

In [ ]:
result.select(F.explode(F.arrays_zip('ner_chunk_generic.result', 'ner_chunk_generic.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-------------------------+---------+
|chunk                    |ner_label|
+-------------------------+---------+
|Michael Berger           |NAME     |
|12 Dezember 2018         |DATE     |
|St. Elisabeth-Krankenhaus|LOCATION |
|Bad Kissingen            |LOCATION |
|Berger                   |NAME     |
|76                       |AGE      |
+-------------------------+---------+



## DeIdentification

### Obfuscation mode

In [4]:
# Downloading custom faker entity list.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/obfuscate.txt

In [ ]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")\

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")\

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)\

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate.txt')\
    .setObfuscateRefSource("file")\

In [ ]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:
deid_lp = LightPipeline(model)

In [ ]:
text = """Michael Berger wird am Morgen des 12 Dezember 2018 ins St. Elisabeth-Krankenhaus in Bad Kissingen eingeliefert. Herr Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen."""

In [ ]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

<PATIENT> wird am Morgen des <DATE> ins St. <HOSPITAL> eingeliefert.
Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.


[************] wird am Morgen des [**************] ins St. [************************************] eingeliefert.
Herr [****] ist ** Jahre alt und hat zu viel Wasser in den Beinen.


**** wird am Morgen des **** ins St. **** eingeliefert.
Herr **** ist **** Jahre alt und hat zu viel Wasser in den Beinen.


Caroline Schuchhardt-Mangold wird am Morgen des 04-14-1998 ins St. Charite Campus Mitte eingeliefert.
Herr Zimmer-Riehl ist 83 Jahre alt und hat zu viel Wasser in den Beinen.


In [ ]:
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame(list(zip(result["masked_with_entity"], result["masked_with_chars"],
                          result["masked_fixed_length_chars"], result["obfuscated"])),
                 columns= ["Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,<PATIENT> wird am Morgen des <DATE> ins St. <HOSPITAL> eingeliefert.,[************] wird am Morgen des [**************] ins St. [************************************...,**** wird am Morgen des **** ins St. **** eingeliefert.,Caroline Schuchhardt-Mangold wird am Morgen des 04-14-1998 ins St. Charite Campus Mitte eingelie...
1,Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.,Herr [****] ist ** Jahre alt und hat zu viel Wasser in den Beinen.,Herr **** ist **** Jahre alt und hat zu viel Wasser in den Beinen.,Herr Zimmer-Riehl ist 83 Jahre alt und hat zu viel Wasser in den Beinen.


### Faker mode

In [ ]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")\

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")\

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)\

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setLanguage('de')\
    .setObfuscateDate(True)\
    .setObfuscateRefSource('faker')

In [ ]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [ ]:
deid_lp = LightPipeline(model)

In [ ]:
text = """Michael Berger wird am Morgen des 12 Dezember 2018 ins St. Elisabeth-Krankenhaus in Bad Kissingen eingeliefert. Herr Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen."""

In [ ]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

<PATIENT> wird am Morgen des <DATE> ins St. <HOSPITAL> eingeliefert.
Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.


[************] wird am Morgen des [**************] ins St. [************************************] eingeliefert.
Herr [****] ist ** Jahre alt und hat zu viel Wasser in den Beinen.


**** wird am Morgen des **** ins St. **** eingeliefert.
Herr **** ist **** Jahre alt und hat zu viel Wasser in den Beinen.


Wendt Herbst wird am Morgen des 06-28-1991 ins St. BELOIT HEALTH SYSTEM eingeliefert.
Herr Peters Bayer ist 80 Jahre alt und hat zu viel Wasser in den Beinen.


In [ ]:
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame(list(zip(result["masked_with_entity"], result["masked_with_chars"],
                          result["masked_fixed_length_chars"], result["obfuscated"])),
                 columns= ["Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,<PATIENT> wird am Morgen des <DATE> ins St. <HOSPITAL> eingeliefert.,[************] wird am Morgen des [**************] ins St. [************************************...,**** wird am Morgen des **** ins St. **** eingeliefert.,Wendt Herbst wird am Morgen des 06-28-1991 ins St. BELOIT HEALTH SYSTEM eingeliefert.
1,Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.,Herr [****] ist ** Jahre alt und hat zu viel Wasser in den Beinen.,Herr **** ist **** Jahre alt und hat zu viel Wasser in den Beinen.,Herr Peters Bayer ist 80 Jahre alt und hat zu viel Wasser in den Beinen.


# 2. Pretrained German Deidentification Pipeline

- We developed a clinical deidentification pretrained pipeline that can be used to deidentify PHI information from German medical texts. The PHI information will be masked and obfuscated in the resulting text. 
- The pipeline can mask and obfuscate:
    - Patient
    - Doctor
    - Hospital
    - Date
    - Organization
    - City
    - Street
    - Country
    - User name
    - Profession
    - Phone
    - Age
    - Contact
    - ID
    - Phone
    - Zip
    - Account
    - SSN
    - Driver's License Number
    - Plate Number

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("clinical_deidentification", "de", "clinical/models")

clinical_deidentification download started this may take some time.
Approx size to download 1.2 GB
[OK!]


In [ ]:
text = """Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus in Bad Kissingen eingeliefert. 
Herr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.

Persönliche Daten :
ID-Nummer: T0110053F
Platte A-BC124
Kontonummer: DE89370400440532013000
SSN : 13110587M565
Lizenznummer: B072RRE2I55
Adresse : St.Johann-Straße 13 19300"""

result = deid_pipeline.annotate(text)
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

Zusammenfassung : [************] wird am Morgen des [**************] ins [***************************************] eingeliefert.
Herr [************] ist ** Jahre alt und hat zu viel Wasser in den Beinen.
Persönliche Daten :
ID-Nummer: [*******]
Platte [*****]
Kontonummer: [********************]
SSN : [**********]
Lizenznummer: [*********]
Adresse : [*****************] [***]


Zusammenfassung : <PATIENT> wird am Morgen des <DATE> ins <HOSPITAL> eingeliefert.
Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.
Persönliche Daten :
ID-Nummer: <ID>
Platte <PLATE>
Kontonummer: <ACCOUNT>
SSN : <SSN>
Lizenznummer: <DLN>
Adresse : <STREET> <ZIP>


Zusammenfassung : **** wird am Morgen des **** ins **** eingeliefert.
Herr **** ist **** Jahre alt und hat zu viel Wasser in den Beinen.
Persönliche Daten :
ID-Nummer: ****
Platte ****
Kontonummer: ****
SSN : ****
Lizenznummer: ****
Adresse : **** ****


Zusammenfassung : Hollmann Burmeister wird am Morgen des 11-01-1970 ins SAN R

In [ ]:
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame(list(zip(result["sentence"], result["masked"],
                          result["masked_with_chars"], result["masked_fixed_length_chars"], result["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhau...,Zusammenfassung : <PATIENT> wird am Morgen des <DATE> ins <HOSPITAL> eingeliefert.,Zusammenfassung : [************] wird am Morgen des [**************] ins [**********************...,Zusammenfassung : **** wird am Morgen des **** ins **** eingeliefert.,Zusammenfassung : Hollmann Burmeister wird am Morgen des 11-01-1970 ins SAN RAMON REGIONAL MEDIC...
1,Herr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.,Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.,Herr [************] ist ** Jahre alt und hat zu viel Wasser in den Beinen.,Herr **** ist **** Jahre alt und hat zu viel Wasser in den Beinen.,Herr Hollmann Burmeister ist 0 Jahre alt und hat zu viel Wasser in den Beinen.
2,Persönliche Daten :\nID-Nummer: T0110053F,Persönliche Daten :\nID-Nummer: <ID>,Persönliche Daten :\nID-Nummer: [*******],Persönliche Daten :\nID-Nummer: ****,Persönliche Daten :\nID-Nummer: L6043236
3,Platte A-BC124,Platte <PLATE>,Platte [*****],Platte ****,Platte QA348G
4,Kontonummer: DE89370400440532013000\nSSN : 13110587M565,Kontonummer: <ACCOUNT>\nSSN : <SSN>,Kontonummer: [********************]\nSSN : [**********],Kontonummer: ****\nSSN : ****,Kontonummer: 192837465738\nSSN : 999-30-4262
5,Lizenznummer: B072RRE2I55,Lizenznummer: <DLN>,Lizenznummer: [*********],Lizenznummer: ****,Lizenznummer: S99913378
6,Adresse : St.Johann-Straße 13 19300,Adresse : <STREET> <ZIP>,Adresse : [*****************] [***],Adresse : **** ****,Adresse : Guntram-Hofmann-Gasse 6 31%05d
